In [ ]:
var openDigger = require('../../src/open_digger');
var now = new Date();
var res = now.setMonth(now.getMonth() - 1); // handle new year
var startYear = 2022, startMonth = 1, endYear = now.getFullYear(), endMonth = now.getMonth() + 1;
var forEveryMonth = f => {
    var index = 0;
    for (var y = startYear; y <= endYear; y++)
        for (var m = 1; m <= (y === endYear ? endMonth : startMonth); m++)
            f(y, m, index++);
};

: 

# X-lab Community Analysis Report

This notebook is used to track the members active and influence status in X-lab community.

The result can be used for community measurement and incentive.

The start date for this report is 2022-01-01 and the end date is up-to-date.

## Overview

This part is the history OpenRank trending of all projects in X-lab community.

In [3]:
openDigger.index.openrank.getRepoOpenrank({
    labelUnion: [':communities/xlab'],
    startYear, startMonth,
    endYear, endMonth,
    groupTimeRange: 'month', limit: -1,
}).then(data => {
    var x = [], v = [], g = []; g.push(0);
    forEveryMonth((y, m, index) => {
        var key = `Y${y}M${m}`;
        x.push(key); v.push(0);
        data.forEach(row => { v[index] += (row.open_rank[index] ? row.open_rank[index] : 0); row[key] = row.open_rank[index] ? row.open_rank[index] : 0; });
        if (index > 0) g.push(Math.ceil((v[index]-v[index-1])*100/v[index-1]));
    });
    data.forEach(row => { delete row.open_rank; delete row.org; });
    console.table(data);
    openDigger.render.plotly([
        { x, y: v, type: 'scatter', name: 'OpenRank' },
        { x, y: g, type: 'bar', yaxis: 'y2', name: 'Growth'}
    ], {
        title: 'X-lab OpenRank trending',
        yaxis2: { overlaying: 'y', side: 'right' }
    });
    fs.writeFileSync('all_data.json', JSON.stringify(data));
});

SyntaxError: invalid syntax (<ipython-input-3-9b89d5409db0>, line 3)

## OpenRank Distribution

The OpenRank distribution for certain month is calculated by every repo with OpenRank in this month and all the developers' OpenRank in these repos in this month.

The algorithm means:

- If you contribute to a high OpenRank repo, your final OpenRank may higher.
- If you contribute more in any repo, your final OpenRank may higher.
- If you have a high OpenRank in a certain repo, make the repo's OpenRank higher, your OpenRank will be higher.

So please contribute more and contribute to the important repos, or make your own repo important.

In [2]:
var calculateOpenRankDistribution = (y, m, base) => {
    var actors = openDigger.label.getGitHubData([':communities/xlab']).githubUsers;
    var key = `Y${y}M${m}`;
    var repos = JSON.parse(fs.readFileSync('all_data.json').toString()).filter(row => row[key] > 0);
    var actorOpenrankMap = new Map();
    var actorLoginMap = new Map();
    var repoArr = [];
    actors.forEach(id => actorOpenrankMap.set(id, 0));
    Promise.all(repos.map(r => {
        return new Promise(resolve => {
            var sql = `MATCH (u:User) WHERE apoc.any.property(u, 'open_rank_${r.id}_${y}${m}') > 0 RETURN u.id AS id, u.login AS login, apoc.any.property(u, 'open_rank_${r.id}_${y}${m}') AS openrank`;
            openDigger.driver.neo4j.query(sql).then(data => {
                var total = 0;
                var devs = [];
                data.forEach(item => total += item.openrank);
                data.filter(item => actors.includes(item.id)).forEach(item => {
                    actorLoginMap.set(item.id, item.login);
                    var openrank = item.openrank / total;
                    actorOpenrankMap.set(item.id, actorOpenrankMap.get(item.id) + r[key] * openrank);
                    devs.push({login: item.login, openrank });
                });
                if (devs.length > 0)
                    repoArr.push({ name: r.repo_name, openrank: r[key], devs: devs.sort((a, b) => b.openrank - a.openrank).map(i => `${i.login}(${(i.openrank*100).toFixed(2)}%)`).join(',')});
                resolve();
            });
        });
    })).then(() => {
        var arr = [];
        var total = 0;
        for (var [ id, openrank ] of actorOpenrankMap.entries()) {
            arr.push({login: actorLoginMap.get(id), openrank});
            total += openrank;
        }
        repoArr.sort((a,b) => b.openrank - a.openrank).forEach(r => console.log(r.name, r.openrank, r.devs));
        console.table(arr.map(i => { return { ...i, ratio: (i.openrank*100/total).toFixed(2) + '%', final: Math.round(i.openrank * base / total) } }).filter(i => i.login).sort((a, b) => b.openrank - a.openrank));
    });
};

## The OpenRank distibution of 2022.6

In [12]:
calculateOpenRankDistribution(2022, 6, 10000);

X-lab2017/OSSDevGov2022 13.25 bifenglin(3.65%)
X-lab2017/open-research 5.65 will-ww(32.92%),birdflyi(19.79%),xiaoya-yaya(11.29%),frank-zsy(5.43%),bifenglin(5.25%),kongzhongsanbu(3.83%),tyn1998(2.26%),wengzhenjie(1.81%),gymgym1212(1.10%)
X-lab2017/open-digger 5.07 frank-zsy(57.63%),xiaoya-yaya(8.39%),xgdyp(1.47%),tyn1998(1.21%),bifenglin(1.21%),gymgym1212(1.06%),chunchill(0.89%),zhicheng-ning(0.89%),soobun(0.89%),wangleo61(0.89%),wengzhenjie(0.89%),kongzhongsanbu(0.89%),birdflyi(0.89%)
hypertrons/hypertrons-crx 4.17 tyn1998(59.84%),zhicheng-ning(4.78%),xgdyp(3.80%),frank-zsy(3.27%),will-ww(1.02%)
X-lab2017/open-wonderland 2.43 will-ww(32.77%),soobun(21.32%),xiaoya-yaya(14.61%),tyn1998(7.63%),zhicheng-ning(7.47%),gymgym1212(6.43%),xgdyp(3.65%)
X-lab2017/2022-SocialComputing 2.4 bifenglin(40.96%)
X-lab2017/DesignThinking-LeanStartup 2.3 zhicheng-ning(31.73%),tyn1998(19.69%),will-ww(5.14%)
X-lab2017/open-leaderboard 1.9 gymgym1212(78.64%),will-ww(11.31%),tyn1998(2.93%),frank-zsy(2.13%)
X-l

In [27]:
calculateOpenRankDistribution(2022, 7, 10000);

X-lab2017/OSSDevGov2022 13.7 zhuxiangning(58.05%),will-ww(8.51%)
X-lab2017/open-digger 9.17 frank-zsy(55.89%),yoyo-wu98(8.35%),longyanz(4.75%),xiaoya-yaya(3.26%),zhuxiangning(2.03%),xgdyp(0.70%),zhicheng-ning(0.51%)
hypertrons/hypertrons-crx 8.18 tyn1998(62.19%),xgdyp(3.92%),zhicheng-ning(3.41%),will-ww(1.21%),frank-zsy(1.11%)
X-lab2017/open-wonderland 5.46 will-ww(34.78%),bifenglin(17.46%),xiaoya-yaya(5.91%),frank-zsy(4.62%),kongzhongsanbu(4.05%),wangleo61(3.96%),yoyo-wu98(3.60%),birdflyi(3.60%),longyanz(2.92%),tyn1998(2.35%),zhicheng-ning(2.33%),xgdyp(2.20%),gymgym1212(1.15%)
X-lab2017/DesignThinking-LeanStartup 4.3 zhicheng-ning(26.27%),tyn1998(16.13%),will-ww(8.61%),soobun(5.78%)
X-lab2017/open-research 3.71 will-ww(29.88%),bifenglin(29.23%),yoyo-wu98(10.80%),xgdyp(4.70%)
X-lab2017/open-leaderboard 1.97 gymgym1212(69.02%),will-ww(30.98%)
X-lab2017/oss101-awesome-list 1.17 bifenglin(45.72%),zhicheng-ning(30.82%),will-ww(23.46%)
X-lab2017/X-lab-website 0.78 zhicheng-ning(80.44%),tyn1

In [3]:
calculateOpenRankDistribution(2022, 8, 10000);

X-lab2017/open-digger 8.53 frank-zsy(46.91%),xiaoya-yaya(6.74%),longyanz(5.87%)
X-lab2017/open-wonderland 6.82 xiaoya-yaya(24.84%),will-ww(18.48%),bifenglin(12.57%),tyn1998(8.48%),zhicheng-ning(8.48%),birdflyi(4.91%),xgdyp(3.44%),wangleo61(2.82%)
hypertrons/hypertrons-crx 6.19 tyn1998(34.64%),frank-zsy(2.79%)
X-lab2017/DesignThinking-LeanStartup 2.97 zhicheng-ning(31.56%),longyanz(11.26%)
X-lab2017/oss101-awesome-list 2.8 bifenglin(36.12%),zhicheng-ning(13.39%),will-ww(11.44%)
X-lab2017/open-perf 0.71 will-ww(70.46%),bifenglin(29.54%)
┌─────────┬─────────────────┬─────────────────────┬──────────┬───────┐
│ (index) │      login      │      openrank       │  ratio   │ final │
├─────────┼─────────────────┼─────────────────────┼──────────┼───────┤
│    0    │   'frank-zsy'   │  4.17400838977852   │ '24.83%' │ 2483  │
│    1    │    'tyn1998'    │  2.722802514916304  │ '16.19%' │ 1619  │
│    2    │  'xiaoya-yaya'  │ 2.2693837140507602  │ '13.50%' │ 1350  │
│    3    │    'will-ww'    │  2.